In [1]:
import llm_core.llm as L
import os

In [2]:
#os.environ['MODEL_PATH'] = "/data/ai_club/llms/qwen2-7b-instruct-q5_k_m.gguf"
LLM_GLOBAL_INSTANCE = L.Llama(
                n_ctx=1000,
                model_path=os.environ['MODEL_PATH'],
                n_gpu_layers=-1, verbose=0,
                # embedding=True
            )

In [3]:
system_input = "You are a Japanese language assistant who speaks in consice, simple sentences."
user_input = "Tell me about your favorite Kanji, in Japanese using Kanji, Hiragana, and Katakana when appropriate"

In [4]:
# No Kanji usage, rare katakana usage
grammar_string = r"""
root ::= sentence{1,2}
allowedchars ::= [\u3040-\u30FF, \u4e00 - \u9faf]
word ::= allowedchars
sentence ::= word particle word desu end
particle ::= "が"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], grammar=my_grammar)

{'id': 'chatcmpl-eedbc656-7d68-4b13-9992-d83a627b3009',
 'object': 'chat.completion',
 'created': 1738601697,
 'model': '/data/ai_club/llms/qwen2-7b-instruct-q5_k_m.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': 'ぼがいです。おがいです。'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 52, 'completion_tokens': 10, 'total_tokens': 62}}

In [7]:
#With basic prompt, no character restrictions spamps \u0000 or \uffff
#Good responses with user_input = "Tell me about your favorite Kanji, in Japanese using Kanji, Hiragana, and Katakana when appropriate"
#Talks in any vocab, but keeps sentences in が + です format. :D
grammar_string = r"""
root ::= sentence{1,2}
allowedchars ::= [\u4e00 - \u9faf, \u3000-\u303f, \u3040-\u309F, \u30a0-\u30ff]
word ::= (unicodechar)+
unicodechar ::= [\u0000-\uffff]
sentence ::= word particle word desu end
particle ::= "が"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], grammar=my_grammar)

{'id': 'chatcmpl-f72712ef-01aa-4ac0-8f4b-96da8685eeed',
 'object': 'chat.completion',
 'created': 1738601928,
 'model': '/data/ai_club/llms/qwen2-7b-instruct-q5_k_m.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '私の最愛の漢字は「豊」です。これは豊かさや多量を表します。豊かな生活を送りたいと思います。読み方は「とよ」です。意味は「多」や「富」を表します。これは幸せで満たされた生活を象徴しています。これからの年間を豊かにするために、この漢字を心に刻みたいと思います。 \n\n(Translation: My favorite Kanji is "豊". It means abundance or richness. I want to live a rich life. It is read as "toyo". It represents \'much\' or \'wealth\'. This Kanji symbolizes a life filled with happiness and fulfillment. I want to inscribe this Kanji in my heart for the upcoming year to enrich it.) \n\nNote: The sentence already includes the Kanji, Hiragana, and Katakana as requested. No additional Katakana is needed since all other components are already in Hiragana or Kanji. If you require pronunciation in Katakana, it is "トヨ". However, the most effective use here is sticking to the provided Japanese 